In [1]:
import tensorflow as tf 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from  tensorflow.keras import models, optimizers, regularizers

In [4]:
model = models.Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,034,177 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Clase 23

train_datagen = ImageDataGenerator(rescale= 1./255,
                            rotation_range = 40,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            shear_range = 0.2,
                            zoom_range = 0.2,
                             horizontal_flip = True
                            )

In [6]:
test_datagen = ImageDataGenerator(rescale= 1./255)

In [13]:
train_generator = train_datagen.flow_from_directory('/kaggle/input/cnn-data-sources/cats_and_dogs/train',
                                 target_size=(150,150),
                                 batch_size=32,
                                 class_mode='binary')
validation_generator = test_datagen.flow_from_directory('/kaggle/input/cnn-data-sources/cats_and_dogs/validation',
                                 target_size=(150,150),
                                 batch_size=32,
                                 class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [11]:
checkpoint = ModelCheckpoint('modelo_perros_gatos.keras', 
                             verbose=1, 
                             monitor='val_accuracy', 
                             save_best_only=True)

In [12]:
model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.Adam(), 
              metrics=['accuracy'])

In [ ]:
hist = model.fit(train_generator, steps_per_epoch=2000//32, 
                 epochs=100, 
                 validation_data=validation_generator,
                 validation_steps=1000//32,
                 callbacks=[checkpoint]
                )

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100


I0000 00:00:1748559380.198614      97 service.cc:148] XLA service 0x7e03040047b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748559380.199484      97 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1748559380.556257      97 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/62 ━━━━━━━━━━━━━━━━━━━━ 6:14 6s/step - accuracy: 0.5938 - loss: 0.6827

I0000 00:00:1748559383.718580      97 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step - accuracy: 0.4864 - loss: 0.9463
Epoch 1: val_accuracy improved from -inf to 0.53629, saving model to modelo_perros_gatos.keras
62/62 ━━━━━━━━━━━━━━━━━━━━ 39s 536ms/step - accuracy: 0.4867 - loss: 0.9434 - val_accuracy: 0.5363 - val_loss: 0.6857
Epoch 2/100
 1/62 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6250 - loss: 0.6898

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy did not improve from 0.53629
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.6250 - loss: 0.6898 - val_accuracy: 0.5020 - val_loss: 0.6864
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 0.5293 - loss: 0.6842
Epoch 3: val_accuracy did not improve from 0.53629
62/62 ━━━━━━━━━━━━━━━━━━━━ 15s 237ms/step - accuracy: 0.5294 - loss: 0.6841 - val_accuracy: 0.5030 - val_loss: 0.6916
Epoch 4/100
 1/62 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4062 - loss: 0.7017
Epoch 4: val_accuracy did not improve from 0.53629
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.4062 - loss: 0.7017 - val_accuracy: 0.5020 - val_loss: 0.6923
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.4969 - loss: 0.6934
Epoch 5: val_accuracy did not improve from 0.53629
62/62 ━━━━━━━━━━━━━━━━━━━━ 14s 234ms/step - accuracy: 0.4972 - loss: 0.6934 - val_accuracy: 0.5202 - val_loss: 0.6893
Epoch 6/100
 1/62 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(hist.history['accuracy'], label = 'Train')
plt.plot(hist.history['val_accuracy'], label = 'Val')
plt.legend()
plt.show()

In [ ]:
test_generator = test_datagen.flow_from_directory('../input/cnn-data-sources/cats_and_dogs/test',
                                 target_size=(150,150),
                                 batch_size=32,
                                 class_mode='binary'
                                 )

In [ ]:
model2 = model

In [ ]:
model2.load_weights('./modelo_perros_gatos.keras')

In [ ]:
model2.evaluate(test_generator)